In [1]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt
import os
from soma import aims
from scipy.signal import find_peaks

## NOISE REMOVAL 
#from skimage.filters.rank import median 
from scipy.ndimage import median_filter, gaussian_filter, generic_filter
from skimage.morphology import disk, ball, cube
from scipy.stats import trim_mean

# Metric extraction 
This notebook sets up extraction of diffusion metrics per tract:

1. DTI metrics need to be pre-cleaned (see notebook 004_bv_clean_DTI.ipynb). Note: L1 == AD, need to compute RD from L2 & L3 maps
2. NODDI metrics need to be pre-cleaned (see notebook 003_bv_NODDI_post_processing.ipynb)
3. Tracts themselves need to be cleaned of spurious streamlines (see notebook 005_bv_tractography_cleanup_thresholds.ipynb)
4. for the metrics where cleaning turned off the voxel, remove the voxel from the tract as well
5. Per tract metric will be computed as: 

$tract metric = \sum (metric_{i} x fibre density_{i}) / \sum fibre density_{i} $ over voxels i of the tract


#### make sure the matching is OK == for controls do not run on all 69 available for the analysis 

In [2]:
def get_tract_label(folder_name):
    label = folder_name.split('2x2_')[-1]
    label = label.split('Mask')
    label = label[0] + label[1] 
    label = label.replace('sh', '')
    return label

def extract_bundle_metric(tract, metric_map):
    
    # remove metric 0 voxels from both arrays   
    tract[np.where(metric_map == 0)] = 0

    # remove tracts 0 voxels from both arrays
    metric_map = metric_map[np.where(tract != 0)]
    tract = tract[np.where(tract != 0)]

    # sanity check 
    if len(metric_map) == len(tract):
        return np.sum(np.multiply(metric_map, tract))/np.sum(tract)
    
    else: 
        print('tract and metric map do not match')
        return np.nan
    
def read_in_DTI_metric_map(subject_id, session_id, metric):
    
    iDir = '/neurospin/grip/external_databases/dHCP_CR_JD_2018/release3/dhcp_dmri_shard_pipeline/sub-{}/ses-{}/dwi/'.format(
                        subject_id, session_id)
    
    if metric == 'FA':
        iPath = os.path.join(iDir, 'DTI/dtifit_b1000/sub-{}_ses-{}_{}.nii.gz'.format(
                                    subject_id, session_id, metric)) 
        metric_map = aims.read(iPath).arraydata()[0]
    
        
    if metric == 'AD':
        iPath = os.path.join(iDir, 'DTI/dtifit_b1000/sub-{}_ses-{}_L1.nii.gz'.format(
                                    subject_id, session_id)) 
        metric_map = aims.read(iPath).arraydata()[0]
        
    elif metric == 'RD':
        iPath2 = os.path.join(iDir, 'DTI/dtifit_b1000/sub-{}_ses-{}_L2.nii.gz'.format(
                                    subject_id, session_id)) 
        iPath3 = os.path.join(iDir, 'DTI/dtifit_b1000/sub-{}_ses-{}_L3.nii.gz'.format(
                                    subject_id, session_id)) 
        a2 = aims.read(iPath2).arraydata()[0]
        a3 = aims.read(iPath3).arraydata()[0]
        
        metric_map = (a2 + a3)/2
        
    elif metric == 'MD':
        iPath1 = os.path.join(iDir, 'DTI/dtifit_b1000/sub-{}_ses-{}_L1.nii.gz'.format(
                                    subject_id, session_id))
        iPath2 = os.path.join(iDir, 'DTI/dtifit_b1000/sub-{}_ses-{}_L2.nii.gz'.format(
                                    subject_id, session_id)) 
        iPath3 = os.path.join(iDir, 'DTI/dtifit_b1000/sub-{}_ses-{}_L3.nii.gz'.format(
                                    subject_id, session_id)) 
        
        a1 = aims.read(iPath1).arraydata()[0]
        a2 = aims.read(iPath2).arraydata()[0]
        a3 = aims.read(iPath3).arraydata()[0]
        
        metric_map =( a1 + a2 + a3)/3
    return metric_map
        
def clean_tracts(tract, threshold):
    limit = np.round(np.max(tract.ravel())*threshold,2)
    tract[tract < limit] = 0
    
    return tract
    

def read_in_NODDI_metric_map(subject_id, session_id, metric):
    
    if metric == 'ODI':
        metric = 'OD'
        
    else:
        metric = 'mean_fintra'

    iDir = '/neurospin/grip/external_databases/dHCP_CR_JD_2018/release3/dhcp_dmri_shard_pipeline/sub-{}/ses-{}/dwi/'.format(
                        subject_id, session_id)
    
    

    iPath = os.path.join(iDir, 'NODDI/{}.nii'.format( metric))
    
    return aims.read(iPath).arraydata()[0]

In [3]:
### NODDI cleaning
def get_groove(metric_map):
    his, bin_edges = np.histogram(metric_map.ravel(), bins=np.arange(0.01,1.01,0.01))
    peaks, _ = find_peaks(his*-1, distance=25)
    first_peak = round(bin_edges[peaks[0]],3)
    
    return first_peak

def calculate_corrected_number(y, first_peak):
    inp = y.copy()
    middleIdx = int(len(inp)/2)
    
    if (0 < inp[middleIdx] < first_peak) or (inp[middleIdx] > 0.95):
        inp.sort()
        inp = inp[(inp >= first_peak) & (inp < 0.95)]
        
        if len(inp) <= 1:
            return 2000

        elif 1 < len(inp) < 3:
            return 1000
        
        else: 
            idx = int(len(inp)/2)
            return 1000
            #return np.mean(inp)
    else: 
        return inp[middleIdx]
    
def alpha_trim_only_outside_range(y, first_peak):
    inp = y.copy()
    middleIdx = int(len(inp)/2)
    
    if (0< inp[middleIdx] < first_peak) or (inp[middleIdx] > 0.95):
        inp.sort()
        inp = inp[(inp >= first_peak) & (inp < 0.95)]
        
        if len(inp) <= 1:
            return np.nan
    
        elif 1 < len(inp) < 3:
            return np.mean(inp)
        
        else: 
            idx = int(len(inp)/2)
            return np.mean(inp[idx-1:idx+2])
            #return np.mean(inp)
    else: 
        return inp[middleIdx]
    
def denoise_NDI(NDI_image, arg):
    im_NDI = NDI_image.copy()
    #new_NDI = NDI_image.copy()
    for i in range(len(arg[0])):
        idx = (arg[0][i], arg[1][i], arg[2][i])
        #print(im_NDI[idx])
        if im_NDI[idx] > 0:
        
    
            cube = im_NDI[idx[0]-1:idx[0]+2,idx[1]-1:idx[1]+2,idx[2]-1:idx[2]+2 ]
            inp = cube.ravel()
            middleIdx = int(len(inp)/2)
            inp.sort()
            if len(inp[middleIdx-1:middleIdx+2]) == 3:
                im_NDI[idx] = np.mean(inp[middleIdx-1:middleIdx+2])
            else:
                im_NDI[idx] = im_NDI[idx] 
            #new_NDI[idx] = np.mean(inp[1:-1])
    
            #print(np.mean(inp[middleIdx-1:middleIdx+2]))
            #print(np.mean(inp[1:-1]))
            #print(inp)
    
        #else: 
         #   new_NDI[idx] = im_NDI[idx]
            #print(im_NDI[idx])
        
    return im_NDI

def clean_NODDI(ODI_vol, NDI_vol):
    
    # get peak 
    first_peak = get_groove(ODI_vol)
    #get cleaning ODI indices
    denoised = generic_filter(ODI_vol, calculate_corrected_number, size=2, extra_arguments=(first_peak,))
    
    ### Correct ODI
    denoised_ODI = generic_filter(ODI_vol, alpha_trim_only_outside_range, size=2, extra_arguments=(first_peak,))
    denoised_ODI[ np.isnan(denoised_ODI)] = 0
    
    ### Correct NDI
    
    arg = np.where(denoised == 1000)
    #print(np.shape(arg[0]))
    denoised_NDI = denoise_NDI(NDI_image = NDI_vol, arg = arg)
    
    #denoised_NDI_turned_off = denoised_NDI.copy()
    #denoised_NDI_turned_off[np.where(denoised == 2000)] = 0
    denoised_NDI[np.where(denoised == 2000)] = 0
    
    return denoised_ODI, denoised_NDI

In [4]:
controls = pd.read_csv('../../DerivedData/control_group_list.csv', header=None)
#preterms = pd.read_csv('../../DerivedData/preterm_group_list.csv', header=None)

In [ ]:
df_DTI = pd.DataFrame()


for i, row in preterms.iterrows():
    print(row[0])
    #df_subj = pd.DataFrame()
    iDir = '/neurospin/grip/external_databases/dHCP_CR_JD_2018/release3/dhcp_dmri_shard_pipeline/sub-{}/ses-{}/dwi/probtrakX_results'.format(
                    row[0], row[1])
    # ceck that the diffusion directory exists 
    if os.path.isdir(iDir):
        # DTI
        for metric in ['FA', 'AD', 'RD', 'MD']:
        
            metric_map = read_in_DTI_metric_map(subject_id=row[0], session_id=row[1], metric=metric)

            folders = [folder for folder in os.listdir(iDir) if 'ROIbyROI' in folder]
            for folder in folders:
                label = get_tract_label(folder)
                #print(label)
        
                iTract = aims.read(os.path.join(iDir, folder, 'fdt_paths.nii.gz'))
                tract = iTract.arraydata()[0]
        
                ### needs tract cleaning here, unless inter-hemispheric, the cleaning thresholds is 5 %
                if label not in ['S1L-S1R', 'M1L-M1R', 'ParacentralL-ParacentralR' ]:
                    tract = clean_tracts(tract=tract, threshold=0.05)
            
                metric_value = extract_bundle_metric(metric_map=metric_map, tract=tract)
            
                #print(row[0], label, metric, metric_value)
            
                df_DTI.loc[i, 'subject_id'] = row[0]
                df_DTI.loc[i, 'session_id'] = int(row[1])
                df_DTI.loc[i, label+'_'+metric] = metric_value
            
        # NODDI 
        ### pre-cleaning 
        for metric in ['NDI', 'ODI']:
            metric_map = read_in_NODDI_metric_map(subject_id=row[0], session_id=row[1], metric = metric)
        
    
            folders = [folder for folder in os.listdir(iDir) if 'ROIbyROI' in folder]
            for folder in folders:
                label = get_tract_label(folder)
                #print(label)
        
                iTract = aims.read(os.path.join(iDir, folder, 'fdt_paths.nii.gz'))
                tract = iTract.arraydata()[0]
        
                ### needs tract cleaning here, unless inter-hemispheric, the cleaning thresholds is 5 %
                if label not in ['S1L-S1R', 'M1L-M1R', 'ParacentralL-ParacentralR' ]:
                    tract = clean_tracts(tract=tract, threshold=0.05)
            
                metric_value = extract_bundle_metric(metric_map=metric_map, tract=tract)
            
                    #print(row[0], label, metric, metric_value)
                
                df_DTI.loc[i, 'subject_id'] = row[0]
                df_DTI.loc[i, 'session_id'] = int(row[1])
                df_DTI.loc[i, label+'_'+metric+'_pre'] = metric_value
        
    
        ## after cleaning 
        ODI_map = read_in_NODDI_metric_map(subject_id=row[0], session_id=row[1], metric = 'ODI')
        NDI_map = read_in_NODDI_metric_map(subject_id=row[0], session_id=row[1], metric = 'NDI')
    
        ODI_map, NDI_map = clean_NODDI(ODI_vol= ODI_map, NDI_vol=NDI_map)
        folders = [folder for folder in os.listdir(iDir) if 'ROIbyROI' in folder]
        for folder in folders:
            label = get_tract_label(folder)
            #print(label)
        
            iTract = aims.read(os.path.join(iDir, folder, 'fdt_paths.nii.gz'))
            tract = iTract.arraydata()[0]
        
            ### needs tract cleaning here, unless inter-hemispheric, the cleaning thresholds is 5 %
            if label not in ['S1L-S1R', 'M1L-M1R', 'ParacentralL-ParacentralR' ]:
                tract = clean_tracts(tract=tract, threshold=0.05)
            
            metric_value_ODI = extract_bundle_metric(metric_map=ODI_map, tract=tract)
        
            #print(label, np.sum(NDI_map))
            metric_value_NDI = extract_bundle_metric(metric_map=NDI_map, tract=tract)

            
                    #print(row[0], label, metric, metric_value)
            
            df_DTI.loc[i, 'subject_id'] = row[0]
            df_DTI.loc[i, 'session_id'] = int(row[1])
            df_DTI.loc[i, label+'_ODI_post'] = metric_value_ODI
            df_DTI.loc[i, label+'_NDI_post'] = metric_value_NDI
    
    
    

CC00830XX14
CC00136AN13
CC00418AN14
CC00136BN13
CC00804XX12
CC00648XX22
CC00802XX10
CC00855XX14
CC00838XX22
CC00997BN25
CC00301XX04
CC00632XX14
CC00919XX20
CC00517XX14
CC00867XX18
CC01011XX05
CC00618XX16
CC00986AN22
CC00829XX21
CC00889BN24
CC00770XX12
CC00628XX18
CC00525XX14
CC00407AN11
CC00135BN12
CC00672BN13
CC00305XX08
CC00526XX15
CC00284AN13
CC00718XX17
CC00271XX08
CC00227XX13
CC00389XX19
CC00621XX11
CC00489XX20
CC00245AN15
CC00797XX23
CC00245BN15
CC00661XX10
CC00518XX15
CC00747XX22
CC00326XX13
CC00686XX19
CC00907XX16
CC00406XX10
CC00529BN18
CC00361XX07
CC00529AN18
CC00955XX15
CC00576XX16
CC00385XX15
CC00407BN11
CC00617XX15
CC00889AN24
CC00191XX11
CC01038XX16
CC00792XX18
CC01005XX07
CC00845AN21
CC00823XX15
CC00845BN21
CC00672AN13
CC01077XX14


In [ ]:
df_DTI['session_id'] = df_DTI['session_id'].astype(int)

In [174]:
df_DTI.to_csv('../../DerivedData/extracted_diffusion_metrics_preterm_group.csv')

# Extract post-processed NODDI mean WM metrics 


In [7]:
controls = pd.read_csv('../../DerivedData/control_group_list.csv', header=None)
#preterms = pd.read_csv('../../DerivedData/preterm_group_list.csv', header=None)

In [8]:
df_WMI = pd.DataFrame()
for i, row in controls.iterrows():
    print(row[0])
    #df_subj = pd.DataFrame()
    iDir = '/neurospin/grip/external_databases/dHCP_CR_JD_2018/release3/dhcp_dmri_shard_pipeline/sub-{}/ses-{}/dwi/probtrakX_results'.format(
                    row[0], row[1])
    # ceck that the diffusion directory exists 
    if os.path.isdir(iDir):
        # NODDI 
        ## after cleaning 
        ODI_map = read_in_NODDI_metric_map(subject_id=row[0], session_id=row[1], metric = 'ODI')
        NDI_map = read_in_NODDI_metric_map(subject_id=row[0], session_id=row[1], metric = 'NDI')
    
        ODI_map, NDI_map = clean_NODDI(ODI_vol= ODI_map, NDI_vol=NDI_map)
        
        iRibbon = '/neurospin/grip/external_databases/dHCP_CR_JD_2018/Projects/rel3_dhcp_dmri_shard_pipeline_analysis/sub-{}/ses-{}/sub-{}_ses-{}_desc-ribbon_dseg_shardDMRI_space.nii.gz'.format(row[0], row[1], row[0], row[1])
    
        wm_ar = aims.read(iRibbon).arraydata()[0]
        
        wm_mask = np.zeros_like(wm_ar)
        wm_mask[wm_ar == 41] = 1
        wm_mask[wm_ar == 2] = 1
        
        ODI_map = ODI_map * wm_mask
        NDI_map = NDI_map * wm_mask
        
        df_WMI.loc[i,'subject_id'] = row[0]
        df_WMI.loc[i, 'ODI_post'] = np.mean(ODI_map[ODI_map != 0])
        df_WMI.loc[i, 'NDI_post'] = np.mean(NDI_map[NDI_map != 0])

CC00939XX24
CC00184XX12
CC00616XX14
CC00801XX09
CC00205XX07
CC00731XX14
CC00550XX06
CC00589XX21
CC00100XX01
CC00852XX11
CC00427XX15
CC01042XX12
CC00247XX17
CC00566XX14
CC00586XX18
CC00498XX21
CC00362XX08
CC00549XX22
CC00071XX06
CC00675XX16
CC00716XX15
CC01198XX20
CC00111XX04
CC00584XX16
CC00150AN02
CC00383XX13
CC00477XX16
CC00865XX16
CC00364XX10
CC00174XX10
CC00540XX13
CC00424XX12
CC00353XX07
CC00500XX05
CC00122XX07
CC00663XX12
CC00667XX16
CC00527XX16
CC00537XX18
CC00713XX12
CC00314XX09
CC00740XX15
CC00082XX09
CC00425XX13
CC00653XX10
CC00485XX16
CC00096XX15
CC00219XX13
CC00502XX07
CC00765XX15
CC00594XX18
CC00798XX24
CC00067XX10
CC00508XX13
CC00265XX10
CC00270XX07
CC00402XX06
CC01014XX08
CC00822XX14
CC00475XX14
CC00516XX13
CC00577XX17
CC00593XX17
CC00091XX10
CC00409XX13
CC00469XX16
CC00178XX14
CC00149XX18
CC00497XX20


In [9]:
df_WMI.to_csv('../../DerivedData/controls_NODDIpost_metrics_over_WM.csv')

#### update the WM tables with post processed NODDI metrics 

In [10]:
controls_WM = pd.read_csv('../../DerivedData/controls_NODDIpost_metrics_over_WM.csv', index_col=0)
preterms_WM = pd.read_csv('../../DerivedData/preterms_NODDIpost_metrics_over_WM.csv', index_col=0)


extreme = pd.read_csv('../../DerivedData/extreme_pairs_mean_diffusion_metrics_over_WM.csv', index_col=0)
moderate = pd.read_csv('../../DerivedData/moderate_pairs_mean_diffusion_metrics_over_WM.csv', index_col=0)

In [11]:
for i, row in extreme.iterrows():
    
    extreme.loc[i, 'preterm_ODI_post'] = preterms_WM[ preterms_WM.subject_id == row.preterm_ID]['ODI_post'].values[0]
    extreme.loc[i, 'preterm_NDI_post'] = preterms_WM[ preterms_WM.subject_id == row.preterm_ID]['NDI_post'].values[0]
    
    extreme.loc[i, 'control_ODI_post'] = controls_WM[ controls_WM.subject_id == row.matched_ID]['ODI_post'].values[0]
    extreme.loc[i, 'control_NDI_post'] = controls_WM[ controls_WM.subject_id == row.matched_ID]['NDI_post'].values[0]
    
extreme.to_csv('../../DerivedData/extreme_pairs_mean_diffusion_metrics_over_WM.csv')


In [12]:
for i, row in moderate.iterrows():
    
    moderate.loc[i, 'preterm_ODI_post'] = preterms_WM[ preterms_WM.subject_id == row.preterm_ID]['ODI_post'].values[0]
    moderate.loc[i, 'preterm_NDI_post'] = preterms_WM[ preterms_WM.subject_id == row.preterm_ID]['NDI_post'].values[0]
    
    moderate.loc[i, 'control_ODI_post'] = controls_WM[ controls_WM.subject_id == row.matched_ID]['ODI_post'].values[0]
    moderate.loc[i, 'control_NDI_post'] = controls_WM[ controls_WM.subject_id == row.matched_ID]['NDI_post'].values[0]
    
moderate.to_csv('../../DerivedData/moderate_pairs_mean_diffusion_metrics_over_WM.csv')


In [13]:
moderate

,preterm_ID,matched_ID,preterm_AD,preterm_RD,preterm_MD,preterm_FA,preterm_NDI,preterm_ODI,control_AD,control_RD,control_MD,control_FA,control_NDI,control_ODI,preterm_ODI_post,preterm_NDI_post,control_ODI_post,control_NDI_post
2,CC00418AN14,CC00616XX14,0.001662,0.001339,0.001447,0.149337,0.119322,0.205933,0.001479,0.001139,0.001252,0.177000,0.176501,0.279964,0.283958,0.123364,0.296826,0.175711
4,CC00804XX12,CC00205XX07,0.001627,0.001311,0.001416,0.149462,0.118283,0.212811,0.001584,0.001262,0.001369,0.154925,0.131754,0.239672,0.278822,0.124492,0.286943,0.134883
5,CC00648XX22,CC00731XX14,0.001471,0.001127,0.001241,0.180050,0.186412,0.283347,0.001480,0.001126,0.001244,0.183334,0.182440,0.276525,0.302956,0.185786,0.303248,0.181561
11,CC00632XX14,CC01042XX12,0.001566,0.001208,0.001327,0.175920,0.169507,0.256374,0.001441,0.001084,0.001203,0.190505,0.188796,0.281850,0.305737,0.167098,0.300090,0.187724
13,CC00517XX14,CC00566XX14,0.001514,0.001148,0.001270,0.186699,0.181084,0.264751,0.001440,0.001083,0.001202,0.190686,0.198703,0.288207,0.283759,0.177971,0.306868,0.197441
17,CC00986AN22,CC00549XX22,0.001610,0.001289,0.001396,0.153385,0.138827,0.236330,0.001432,0.001085,0.001201,0.186315,0.198306,0.295252,0.285837,0.142142,0.313725,0.196809
18,CC00829XX21,CC00071XX06,0.001646,0.001278,0.001401,0.170779,0.134129,0.212502,0.001471,0.001132,0.001245,0.176149,0.169592,0.280767,0.284400,0.137272,0.302156,0.169273
19,CC00889BN24,CC00675XX16,0.001626,0.001290,0.001402,0.157904,0.133529,0.222688,0.001543,0.001208,0.001320,0.163930,0.157506,0.264026,0.294135,0.138038,0.282790,0.155281
23,CC00407AN11,CC00584XX16,0.001583,0.001245,0.001358,0.161700,0.154603,0.262616,0.001380,0.001039,0.001153,0.188021,0.205924,0.309306,0.302685,0.152905,0.318390,0.205609
24,CC00135BN12,CC00150AN02,0.001601,0.001235,0.001357,0.175889,0.140893,0.211492,0.001608,0.001273,0.001385,0.159358,0.125338,0.208991,0.268408,0.140902,0.263229,0.129153


# Save 2 example post-processed subjects for paper 
FT: CC00529AN18 <br>
PT: CC00731XX14

In [23]:
controls = pd.read_csv('../../DerivedData/control_group_list.csv', header=None)
preterms = pd.read_csv('../../DerivedData/preterm_group_list.csv', header=None)

subjects = pd.concat([controls, preterms])
subjects = subjects[subjects[0].isin(['CC00529AN18', 'CC00731XX14'])]

In [31]:

for i, row in subjects.iterrows():
    print(row[0])
    #df_subj = pd.DataFrame()
    iDir = '/neurospin/grip/external_databases/dHCP_CR_JD_2018/release3/dhcp_dmri_shard_pipeline/sub-{}/ses-{}/dwi/probtrakX_results'.format(
                    row[0], row[1])
    # ceck that the diffusion directory exists 
    if os.path.isdir(iDir):
                  
        # NODDI 

    
        ## after cleaning 
        ODI_map = read_in_NODDI_metric_map(subject_id=row[0], session_id=row[1], metric = 'ODI')
        NDI_map = read_in_NODDI_metric_map(subject_id=row[0], session_id=row[1], metric = 'NDI')
    
        ODI_map, NDI_map = clean_NODDI(ODI_vol= ODI_map, NDI_vol=NDI_map)
       
        ### save ODI map 
        
        ODI_path = '/neurospin/grip/external_databases/dHCP_CR_JD_2018/release3/dhcp_dmri_shard_pipeline/sub-{}/ses-{}/dwi/NODDI/OD.nii'.format(
                        row[0], row[1])
        ODI_pre = aims.read(ODI_path)
        
        new_im = aims.Volume(ODI_map)
        new_im.header().update(ODI_pre.header())

        aims.write(new_im, '../../Results/figures/{}_{}_ODI_post_processed.nii'.format(row[0], row[1]))
        
        if os.path.isfile('../../Results/figures/{}_{}_ODI_post_processed.nii'.format(row[0], row[1])):
                print('{} {} {} metric finished'.format(row[0], row[1], 'ODI'))
        
        ### save NDI map

        NDI_path = '/neurospin/grip/external_databases/dHCP_CR_JD_2018/release3/dhcp_dmri_shard_pipeline/sub-{}/ses-{}/dwi/NODDI/mean_fintra.nii'.format(
                        row[0], row[1])
        NDI_pre = aims.read(ODI_path)
        
        new_im = aims.Volume(NDI_map)
        new_im.header().update(NDI_pre.header())

        aims.write(new_im, '../../Results/figures/{}_{}_NDI_post_processed.nii'.format(row[0], row[1]))
        
        if os.path.isfile('../../Results/figures/{}_{}_NDI_post_processed.nii'.format(row[0], row[1])):
                print('{} {} {} metric finished'.format(row[0], row[1], 'NDI'))

CC00731XX14
CC00731XX14 214500 ODI metric finished
CC00731XX14 214500 NDI metric finished
CC00529AN18
CC00529AN18 170000 ODI metric finished
CC00529AN18 170000 NDI metric finished
